In [40]:
from PIL import Image
import numpy as np

In [48]:
THRESHOLD_CONSTANT = 4 #Number used to scale secret image to test if carry image can contain secret image
CARRY_IMAGE_FILE = "D:/Images/canyon1.png"
MESG_IMAGE_FILE = "D:/Images/hello.png"

In [49]:
#This tests each image in array format
#Provides feedback given lengths of array
#Good since dimensions doesn't really matter when hiding an image,
#   we need to know if we have enough pixels to hide the image, not necessarily
#   the dimensions
def img_cmp_arr(carry, message):

    carry_im = Image.open(carry)
    mesg_im = Image.open(message)

    carry_arr = list(carry_im.getdata())
    mesg_arr = list(mesg_im.getdata())

    carry_size = len(carry_arr)
    mesg_size = len(mesg_arr)

    if (carry_size > (THRESHOLD_CONSTANT*(mesg_size))):
        return True
    else:
        return False

In [50]:
def integer_to_binary(rgb):
    '''
    Convert RGB pixel values from integer to binary
    INPUT: An integer tuple (e.g. (220, 110, 96))
    OUTPUT: A string tuple (e.g. ("00101010", "11101011", "00010110"))
    '''
    r, g, b = rgb
    return ('{0:08b}'.format(r),
            '{0:08b}'.format(g),
            '{0:08b}'.format(b))

In [51]:
def binary_to_integer(rgb):
    '''
    Convert RGB pixel values from binary to integer.
    INPUT: A string tuple (e.g. ("00101010", "11101011", "00010110"))
    OUTPUT: Return an int tuple (e.g. (220, 110, 96))
    '''
    r, g, b = rgb
    return (int(r, 2),
            int(g, 2),
            int(b, 2))

In [52]:
def merge_rgb(rgb1, rgb2):
    '''
    Merge two RGB pixels using 4 least significant bits.
    INPUT: A string tuple (e.g. ("00101010", "11101011", "00010110")),
           Another string tuple (e.g. ("00101010", "11101011", "00010110"))
    OUTPUT: An integer tuple with the two RGB values merged
    '''
    r1, g1, b1 = rgb1
    r2, g2, b2 = rgb2
    rgb = (r1[:4] + r2[:4],
           g1[:4] + g2[:4],
           b1[:4] + b2[:4])
    return rgb

In [57]:
def mergeImages(carryImage_path, mesgImage_path):
    '''
    Merge two images. The msegImage will be merged into the carryImage.
    INPUT: carry and message image path
    OUTPUT: A new merged image.
    '''   
    carry_image = Image.open(carryImage_path)
    message_image = Image.open(mesgImage_path)
    
    # Ensure carry image is larger than message image
    if not img_cmp_arr(carryImage_path,mesgImage_path):
       raise ValueError('Carry image size is lower than message image size!')
    
    # Create a new image that will be outputted
    new_image = Image.new(carry_image.mode, carry_image.size)
    pixels_new_arr = list(new_image.getdata())
    
    carry_arr = list(carry_image.getdata())
    mesg_arr = list(message_image.getdata())
    
    for i in range(len(carry_arr)):
        rgb1 = integer_to_binary(carry_arr[i])

        # Use a black pixel as default
        rgb2 = integer_to_binary((0, 0, 0))

        # Check if the pixel count is valid for the second image
        if i < len(mesg_arr):
            rgb2 = integer_to_binary(mesg_arr[i])

        # Merge the two pixels and convert it to a integer tuple
        rgb = merge_rgb(rgb1, rgb2)

        pixels_new_arr[i] = binary_to_integer(rgb)
    
        new_image.convert('RGB').save('merged.png')
    
    return new_image
    
    

In [59]:
#mergeImages(CARRY_IMAGE_FILE,MESG_IMAGE_FILE)